In [8]:
import json
from itertools import combinations
from collections import defaultdict

In [19]:
def find_groups_with_unique_techniques(json_path):
    """
    Identifies groups that have techniques not used by any other group,
    and prints technique IDs along with their names.

    Parameters
    ----------
    json_path : str
        Path to the JSON file containing techniques_map.

    Returns
    -------
    dict
        Contains per-group unique techniques and total globally unique techniques.
    """
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Reverse map: technique_id → set of group_ids
    technique_to_groups = defaultdict(set)

    for group_id, info in data.items():
        for tech_id in info.get("items", []):
            technique_to_groups[tech_id].add(group_id)

    group_results = []
    globally_unique_techniques = set()

    for tech_id, groups in technique_to_groups.items():
        if len(groups) == 1:
            globally_unique_techniques.add(tech_id)

    for group_id, info in data.items():
        group_name = info.get("name", "")
        techniques = info.get("items", [])
        technique_names = info.get("technique name", [])
        tech_map = dict(zip(techniques, technique_names))

        unique_techniques = [
            (t, tech_map.get(t, "<unknown>")) for t in techniques if t in globally_unique_techniques
        ]

        if unique_techniques:
            group_results.append((group_id, group_name, unique_techniques))

    # Output summary
    print(f"Total groups with unique techniques: {len(group_results)}")
    print(f"Total unique techniques across all groups: {len(globally_unique_techniques)}\n")

    for group_id, group_name, techniques in group_results:
        print(f"Group: {group_id} ({group_name})")
        print(f"  Unique Techniques ({len(techniques)}):")
        for tid, tname in techniques:
            print(f"    - {tid}: {tname}")
        print()

    return {
        "groups_with_uniques": group_results,
        "total_unique_techniques": len(globally_unique_techniques),
        "globally_unique_techniques": globally_unique_techniques
    }


In [20]:
# Example usage
json_path = "mitre_techniques_map.json"  # Replace with actual path
groups_with_uniques = find_groups_with_unique_techniques(json_path)

Total groups with unique techniques: 52
Total unique techniques across all groups: 147

Group: G0001 (Axiom)
  Unique Techniques (3):
    - T1563.002: RDP Hijacking
    - T1001.002: Steganography
    - T1553: Subvert Trust Controls

Group: G0004 (Ke3chang)
  Unique Techniques (1):
    - T1558.001: Golden Ticket

Group: G0005 (APT12)
  Unique Techniques (1):
    - T1568.003: DNS Calculation

Group: G0007 (APT28)
  Unique Techniques (8):
    - T1550.001: Application Access Token
    - T1092: Communication Through Removable Media
    - T1546.015: Component Object Model Hijacking
    - T1048.002: Exfiltration Over Asymmetric Encrypted Non-C2 Protocol
    - T1211: Exploitation for Defense Evasion
    - T1001.001: Junk Data
    - T1498: Network Denial of Service
    - T1137.002: Office Test

Group: G0010 (Turla)
  Unique Techniques (3):
    - T1615: Group Policy Discovery
    - T1546.013: Process Discovery
    - T1584.003: Visual Basic

Group: G0012 (Darkhotel)
  Unique Techniques (1):
    -

In [21]:
def find_unique_technique_pairs_with_names(json_path):
    """
    Analyze groups to find unique technique pairs (after removing globally unique techniques),
    and print both technique IDs and names.

    Parameters
    ----------
    json_path : str
        Path to JSON file containing techniques_map.

    Returns
    -------
    dict
        Summary of analysis, including groups with unique pairs and newly discovered unique pair groups.
    """
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Build technique-to-groups mapping
    technique_to_groups = defaultdict(set)
    for group_id, info in data.items():
        for tech in info.get("items", []):
            technique_to_groups[tech].add(group_id)

    # Identify globally unique techniques
    globally_unique_techniques = {tech for tech, groups in technique_to_groups.items() if len(groups) == 1}

    # Build pair-to-groups and group-to-pairs mappings
    pair_to_groups = defaultdict(set)
    group_to_pairs = dict()

    for group_id, info in data.items():
        techs = info.get("items", [])
        tech_names = info.get("technique name", [])
        tech_map = dict(zip(techs, tech_names))

        filtered_techs = [t for t in techs if t not in globally_unique_techniques]
        pairs = list(combinations(sorted(filtered_techs), 2))
        group_to_pairs[group_id] = {
            "pairs": pairs,
            "tech_map": tech_map,
            "name": info.get("name", "")
        }

        for pair in pairs:
            pair_to_groups[pair].add(group_id)

    # Identify unique pairs (appear in only one group)
    unique_pairs = {pair for pair, groups in pair_to_groups.items() if len(groups) == 1}

    # Find which groups contain at least one unique pair
    group_with_unique_pairs = defaultdict(list)
    for group_id, pair_info in group_to_pairs.items():
        for pair in pair_info["pairs"]:
            if pair in unique_pairs:
                group_with_unique_pairs[group_id].append(pair)

    # Compare to groups that already had globally unique techniques
    groups_with_unique_techs = {group_id for group_id, info in data.items()
                                 if any(t in globally_unique_techniques for t in info.get("items", []))}
    new_groups_with_unique_pairs = set(group_with_unique_pairs) - groups_with_unique_techs

    # Print Summary
    print(f"\nTotal groups with globally unique techniques: {len(groups_with_unique_techs)}")
    print(f"Total groups with unique technique pairs (excluding globals): {len(group_with_unique_pairs)}")
    print(f"New groups with unique technique pairs only: {len(new_groups_with_unique_pairs)}\n")

    for group_id in sorted(group_with_unique_pairs):
        info = group_to_pairs[group_id]
        tech_map = info["tech_map"]
        print(f"Group: {group_id} ({info['name']})")
        for t1, t2 in group_with_unique_pairs[group_id]:
            print(f"  - ({t1}, {t2}) → \"{tech_map.get(t1)}\" + \"{tech_map.get(t2)}\"")
        print()

    return {
        "groups_with_unique_pairs": group_with_unique_pairs,
        "new_unique_pair_groups": new_groups_with_unique_pairs,
        "all_unique_pairs": unique_pairs
    }




In [22]:
result = find_unique_technique_pairs_with_names(json_path)


Total groups with globally unique techniques: 52
Total groups with unique technique pairs (excluding globals): 112
New groups with unique technique pairs only: 63

Group: G0001 (Axiom)
  - (T1003, T1546.008) → "OS Credential Dumping" + "Accessibility Features"
  - (T1003, T1566) → "OS Credential Dumping" + "Phishing"
  - (T1003, T1583.002) → "OS Credential Dumping" + "DNS Server"
  - (T1003, T1584.005) → "OS Credential Dumping" + "Botnet"
  - (T1005, T1566) → "Data from Local System" + "Phishing"
  - (T1005, T1583.002) → "Data from Local System" + "DNS Server"
  - (T1021.001, T1566) → "Remote Desktop Protocol" + "Phishing"
  - (T1021.001, T1584.005) → "Remote Desktop Protocol" + "Botnet"
  - (T1078, T1566) → "Valid Accounts" + "Phishing"
  - (T1078, T1583.002) → "Valid Accounts" + "DNS Server"
  - (T1189, T1546.008) → "Drive-by Compromise" + "Accessibility Features"
  - (T1189, T1566) → "Drive-by Compromise" + "Phishing"
  - (T1189, T1583.002) → "Drive-by Compromise" + "DNS Server"
  

In [31]:
def find_unique_technique_triplets_with_names(json_path):
    """
    Analyze groups to find unique technique triplets (excluding globally unique techniques and pairs),
    and print both technique IDs and names.

    Parameters
    ----------
    json_path : str
        Path to JSON file containing techniques_map.

    Returns
    -------
    dict
        Summary of analysis, including groups with unique triplets and newly discovered unique triplet groups.
    """
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Step 1: Build technique-to-groups mapping
    technique_to_groups = defaultdict(set)
    for group_id, info in data.items():
        for tech in info.get("items", []):
            technique_to_groups[tech].add(group_id)

    # Step 2: Identify globally unique techniques
    globally_unique_techniques = {tech for tech, groups in technique_to_groups.items() if len(groups) == 1}

    # Step 3: Identify globally unique technique pairs
    pair_to_groups = defaultdict(set)
    group_to_pairs = dict()

    for group_id, info in data.items():
        techs = [t for t in info.get("items", []) if t not in globally_unique_techniques]
        pairs = list(combinations(sorted(techs), 2))
        group_to_pairs[group_id] = pairs
        for pair in pairs:
            pair_to_groups[pair].add(group_id)

    unique_pairs = {pair for pair, groups in pair_to_groups.items() if len(groups) == 1}
    unique_pair_techniques = set(t for pair in unique_pairs for t in pair)

    # Step 4: Build triplet-to-groups and group-to-triplets mappings
    triplet_to_groups = defaultdict(set)
    group_to_triplets = dict()

    for group_id, info in data.items():
        techs = info.get("items", [])
        tech_names = info.get("technique name", [])
        tech_map = dict(zip(techs, tech_names))
        filtered_techs = [
            t for t in techs
            if t not in globally_unique_techniques and t not in unique_pair_techniques
        ]
        triplets = list(combinations(sorted(filtered_techs), 3))
        group_to_triplets[group_id] = {
            "triplets": triplets,
            "tech_map": tech_map,
            "name": info.get("name", "")
        }
        for triplet in triplets:
            triplet_to_groups[triplet].add(group_id)

    # Step 5: Identify unique triplets (appear in only one group)
    unique_triplets = {triplet for triplet, groups in triplet_to_groups.items() if len(groups) == 1}

    # Step 6: Find which groups contain at least one unique triplet
    groups_with_unique_triplets = defaultdict(list)
    for group_id, triplet_info in group_to_triplets.items():
        for triplet in triplet_info["triplets"]:
            if triplet in unique_triplets:
                groups_with_unique_triplets[group_id].append(triplet)

    # Compare to groups that already had globally unique techniques or unique pairs
    groups_with_unique_techs = {group_id for group_id, info in data.items()
                                 if any(t in globally_unique_techniques for t in info.get("items", []))}
    groups_with_unique_pairs = {group_id for group_id, pairs in group_to_pairs.items()
                                 if any(p in unique_pairs for p in pairs)}
    previously_identified_groups = groups_with_unique_techs.union(groups_with_unique_pairs)

    new_unique_triplet_groups = set(groups_with_unique_triplets) - previously_identified_groups

    # Step 7: Print summary
    print(f"Total groups with globally unique techniques: {len(groups_with_unique_techs)}")
    print(f"Total groups with unique technique pairs: {len(groups_with_unique_pairs)}")
    print(f"Total groups with unique technique triplets (excluding above): {len(groups_with_unique_triplets)}")
    print(f"New groups with only unique triplets: {len(new_unique_triplet_groups)}\n")

    for group_id in sorted(groups_with_unique_triplets):
        info = group_to_triplets[group_id]
        tech_map = info["tech_map"]
        print(f"Group: {group_id} ({info['name']})")
        for t1, t2, t3 in groups_with_unique_triplets[group_id]:
            print(f"  - ({t1}, {t2}, {t3}) → \"{tech_map.get(t1)}\" + \"{tech_map.get(t2)}\" + \"{tech_map.get(t3)}\"")
        print()

    return {
        "groups_with_unique_triplets": groups_with_unique_triplets,
        "new_unique_triplet_groups": new_unique_triplet_groups,
        "all_unique_triplets": unique_triplets
    }


In [30]:
find_unique_technique_triplets_with_names(json_path)


🔹 Total groups with globally unique techniques: 52
🔹 Total groups with unique technique pairs: 112
🔍 Total groups with unique technique triplets (excluding above): 0
🆕 New groups with only unique triplets: 0



{'groups_with_unique_triplets': defaultdict(list, {}),
 'new_unique_triplet_groups': set(),
 'all_unique_triplets': set()}